In [1]:
import os
import nltk
from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()
from nltk.tokenize import RegexpTokenizer
from pymorphy2.tokenizers import simple_word_tokenize
tokenizer = RegexpTokenizer(r'\w+')

import RAKE
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('russian')
rake = RAKE.Rake(stop)
from summa import keywords
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.summarization import keywords as kw


def normalize_text(text):
    text = tokenizer.tokenize(text.lower())
    lemmas = [m.parse(t)[0].normal_form for t in text]
    return ' '.join(lemmas)


def normalize(text):
    text = tokenizer.tokenize(text.lower())
    return [m.parse(t)[0].normal_form for t in text]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\qwe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
with open('./texts/meta.txt', 'r', encoding='utf-8') as f:
    meta = [normalize(line) for line in f.read().split('\n')]
meta

[['индивидуальный',
  'различие',
  'характеристика',
  'признак',
  'психология',
  'причина',
  'личность'],
 ['психологический',
  'тип',
  'типология',
  'характеристика',
  'различие',
  'группа',
  'характер'],
 ['черта',
  'психологический',
  'характеристика',
  'отличие',
  'концептуализация',
  'свойство',
  'классификация'],
 ['интеллект',
  'исследование',
  'тип',
  'показатель',
  'информация',
  'память',
  'характеристика'],
 ['личность',
  'темперамент',
  'эмоциональность',
  'поведение',
  'характеристика',
  'тип',
  'характер']]

In [3]:
data = []
for file in os.listdir('./texts'):
    if file.startswith('text'):
        path = os.path.join('./texts/', file)
        with open(path, 'r', encoding='utf-8') as f:
            nb = int(file[4])
            data.append(tuple([f.read(), meta[nb-1]]))

corpus = [d[0] for d in data]       

In [4]:
data = []
for file in os.listdir('./texts'):
    if file.startswith('text'):
        path = os.path.join('./texts/', file)
        with open(path, 'r', encoding='utf-8') as f:
            data.append(tuple([f.read(), meta[nb-1]]))

In [5]:
def textRank(text):
    text = normalize_text(text)
    result = []
    kw(text, pos_filter=[], scores=True)
    keywords.keywords(normalize_text(text), additional_stopwords=stop, scores=True)
    G = keywords.get_graph(text)
    result = [(edge, G.edge_weight(edge)) for edge in G.edges() if all(map(lambda x: x not in stop, edge))]
    result = sorted(result, key = lambda x: x[1], reverse=True)[:7]
    for i in result:
        print(i)
    return result

In [6]:
 def TfIdf(text, ng = (1,3)):
    vectorizer = TfidfVectorizer(ngram_range = ng)
    X = vectorizer.fit_transform([normalize_text(text)])
    names = vectorizer.get_feature_names()
    words = {}
    for col in X.nonzero()[1]:
        if names[col] not in stop:
            words[names[col]] =  float(X[0, col])
    top = sorted(words.items(), key = lambda x: x[1], reverse=True)
    for i in top[:5]:
        print(i[0], i[1])
    return [i[0] for i in top[:7]]

In [7]:
def Rake(text):
    pred = []
    for i in rake.run(normalize_text(text), minFrequency=2, maxWords=3)[:7]:
            print(i[0], i[1])
            pred.append(i[0])
    return pred

In [8]:
textrank_res = []
for d in corpus:
    textrank_res.append(textRank(d))
    print('============')

(('предмет', 'психология'), 1)
(('психология', 'предмет'), 1)
(('индивидуальный', 'психология'), 1)
(('психология', 'индивидуальный'), 1)
(('различие', 'индивидуальный'), 1)
(('индивидуальный', 'различие'), 1)
(('определять', 'различие'), 1)
(('глава', 'психологический'), 1)
(('психологический', 'глава'), 1)
(('тип', 'психологический'), 1)
(('психологический', 'тип'), 1)
(('настроение', 'тип'), 1)
(('тип', 'настроение'), 1)
(('реакция', 'быстрота'), 1)
(('индивидуальность', 'элемент'), 1)
(('элемент', 'индивидуальность'), 1)
(('эволюция', 'индивидуальность'), 1)
(('индивидуальность', 'эволюция'), 1)
(('типологический', 'эволюция'), 1)
(('эволюция', 'типологический'), 1)
(('теория', 'типологический'), 1)
(('когнитивный', 'теория'), 1)
(('теория', 'когнитивный'), 1)
(('интеллект', 'теория'), 1)
(('теория', 'интеллект'), 1)
(('когнитивный', 'интеллект'), 1)
(('интеллект', 'когнитивный'), 1)
(('предполагать', 'интеллект'), 1)
(('глава', 'темперамент'), 1)
(('темперамент', 'глава'), 1)
(('н

In [9]:
tfidf_res = []
for d in corpus:
    tfidf_res.append(TfIdf(d, ng = (3,3)))
    print('==================')

психология индивидуальный различие 0.3564060541537692
диапазон индивидуальный различие 0.2036606023735824
индивидуальный различие идиографический 0.0763727258900934
формирование индивидуальный различие 0.0763727258900934
проблема психология индивидуальный 0.0763727258900934
он душевный склад 0.06516352124510746
нрав нынешний век 0.06516352124510746
или нрав нынешний 0.06516352124510746
характер или нрав 0.06516352124510746
он говорить что 0.06516352124510746
для тот чтобы 0.15359390551474136
друг от друг 0.10239593700982758
тот свойство который 0.07679695275737068
исследование индивидуальный различие 0.07679695275737068
который он называть 0.07679695275737068
теория тройственный интеллект 0.07992189231706229
способность управлять свой 0.05328126154470819
психометрический показатель интеллект 0.05328126154470819
реакция психофизиологический показатель 0.05328126154470819
время реакция психофизиологический 0.05328126154470819
выделение свойство темперамент 0.08755626081266685
утверждение

In [10]:
for d in corpus:
    TfIdf(d, ng = (1,3))
    print('==================')

различие 0.3608345599161033
индивидуальный 0.3147705735438348
индивидуальный различие 0.2917385803577005
психологический 0.2533519250474768
характеристика 0.21496526973725302
человек 0.2936708755356073
психологический 0.1644556902999401
характеристика 0.1527088552785158
свой 0.12921518523566722
характер 0.10572151519281862
который 0.2175522753468155
человек 0.2175522753468155
черта 0.20918488014116876
свойство 0.17571529931858174
индивидуальный 0.167347904112935
интеллект 0.6076344011667276
интеллектуальный 0.17138406186753857
теория 0.1635938772371959
время 0.14801350797651058
показатель 0.1012724001944546
темперамент 0.4413249612905585
реакция 0.1863372058782358
характеристика 0.1863372058782358
свойство 0.16672276315421097
особенность 0.16672276315421097


In [11]:
tfidf_res = []
for d in corpus:
    tfidf_res.append(TfIdf(d, ng = (2,2)))
    print('==================')

индивидуальный различие 0.6359888306543495
психологический характеристика 0.31799441532717476
психология индивидуальный 0.23431167445160245
диапазон индивидуальный 0.1338923854009157
различие между 0.1338923854009157
психологический характеристика 0.18148518120567103
индивидуальный различие 0.1209901208037807
психологический тип 0.1209901208037807
характер феофраста 0.09074259060283552
поведение человек 0.09074259060283552
индивидуальный различие 0.3008839939729252
на основание 0.21491713855208944
способ выделение 0.12895028313125367
тот чтобы 0.12895028313125367
для тот 0.12895028313125367
время реакция 0.18060945639429235
время опознание 0.1580332743450058
интеллектуальный деятельность 0.1580332743450058
обработка информация 0.1580332743450058
показатель интеллект 0.13545709229571926
свойство темперамент 0.35411844351988625
характеристика темперамент 0.1517650472228084
особенность темперамент 0.12647087268567367
реакция на 0.10117669814853893
реакция время 0.0758825236114042


In [12]:
rake_res = []
for d in data:
    rake_res.append(Rake(d[0]))
    print('==================')

психологический характеристика 4.809523809523809
человек 2.2
который 2.090909090909091
мочь 1.6666666666666667
весь 1.5
группа 1.5
похожий 1.5
человек 2.076923076923077
друг 2.0
группа 1.8571428571428572
мочь 1.75
камень 1.6666666666666667
понять 1.3333333333333333
представитель 1.0
понять индивидуальный различие 8.083333333333334
разный ситуация 4.5
выделить свойство 4.285714285714286
собственный опыт 4.0
общий психология 4.0
среднее ребёнок 4.0
человек оказываться 3.857142857142857
маленький время опознание 8.857142857142858
связь скоростной характеристика 8.323809523809523
выше интеллект 4.25
интеллект 2.25
который 2.25
связать 1.8
работа 1.3333333333333333
разный автор 4.45
темперамент 2.2142857142857144
содержание 1.75
ребёнок 1.6666666666666667
интенсивность 1.0
е 1.0
затем 1.0


In [13]:
print(tfidf_res, '\n', textrank_res, '\n', rake_res)

[['индивидуальный различие', 'психологический характеристика', 'психология индивидуальный', 'диапазон индивидуальный', 'различие между', 'психологический особенность', 'какой то'], ['психологический характеристика', 'индивидуальный различие', 'психологический тип', 'характер феофраста', 'поведение человек', 'том далее', 'не только'], ['индивидуальный различие', 'на основание', 'способ выделение', 'тот чтобы', 'для тот', 'представлять себя', 'том далее'], ['время реакция', 'время опознание', 'интеллектуальный деятельность', 'обработка информация', 'показатель интеллект', 'когнитивный теория', 'интеллектуальный сфера'], ['свойство темперамент', 'характеристика темперамент', 'особенность темперамент', 'реакция на', 'реакция время', 'оцениваться по', 'стилевой особенность']] 
 [[(('предмет', 'психология'), 1), (('психология', 'предмет'), 1), (('индивидуальный', 'психология'), 1), (('психология', 'индивидуальный'), 1), (('различие', 'индивидуальный'), 1), (('индивидуальный', 'различие'), 1)

In [14]:
from sklearn.metrics import f1_score
metrics_true = []
y_pred = []
for line in meta:
    metrics_true.append([1] * len(line))
    y_pred.append([0] * len(line))
(metrics_true, y_pred)

([[1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1]],
 [[0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0]])

In [15]:
from copy import deepcopy

In [16]:
rake_pred = deepcopy(y_pred)
tfidf_pred = deepcopy(y_pred)
textrank_pred = deepcopy(y_pred)

### TfIdf

In [17]:
for nb,text in enumerate(tfidf_res):
    print(nb, text, meta[nb])
    for idx, words in enumerate(text):
        line = words.split()
        tfidf_pred[nb][idx] = int(True in (map(lambda x: x in meta[nb], line)))
tfidf_pred            

0 ['индивидуальный различие', 'психологический характеристика', 'психология индивидуальный', 'диапазон индивидуальный', 'различие между', 'психологический особенность', 'какой то'] ['индивидуальный', 'различие', 'характеристика', 'признак', 'психология', 'причина', 'личность']
1 ['психологический характеристика', 'индивидуальный различие', 'психологический тип', 'характер феофраста', 'поведение человек', 'том далее', 'не только'] ['психологический', 'тип', 'типология', 'характеристика', 'различие', 'группа', 'характер']
2 ['индивидуальный различие', 'на основание', 'способ выделение', 'тот чтобы', 'для тот', 'представлять себя', 'том далее'] ['черта', 'психологический', 'характеристика', 'отличие', 'концептуализация', 'свойство', 'классификация']
3 ['время реакция', 'время опознание', 'интеллектуальный деятельность', 'обработка информация', 'показатель интеллект', 'когнитивный теория', 'интеллектуальный сфера'] ['интеллект', 'исследование', 'тип', 'показатель', 'информация', 'память', 

[[1, 1, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 1, 0, 0],
 [1, 1, 1, 0, 0, 0, 0]]

In [18]:
y_true = [item for inner in metrics_true for item in inner]
y_pred = [item for inner in tfidf_pred for item in inner]
# sklearn.metrics.accuracy_score(y_true, y_pred), sklearn.metrics.recall_score(y_true, y_pred), f1_score(y_true, y_pred) 
m = sklearn.metrics.precision_recall_fscore_support(y_true, y_pred)
print('\t'.join([str(item) for item in m]))

[0. 1.]	[0.  0.4]	[0.         0.57142857]	[ 0 35]


c:\users\qwe\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


#### Ошибки

* Не выкидывает местоимения (какой-то), нужно их включать в стоп-слова
* Самые частотные != ключевые

### Rake

In [19]:
for nb,text in enumerate(rake_res):
    print(nb, text, meta[nb])
    for idx, words in enumerate(text):
        line = words.split()
        rake_pred[nb][idx] = int(True in (map(lambda x: x in meta[nb], line)))
rake_pred  

0 ['психологический характеристика', 'человек', 'который', 'мочь', 'весь', 'группа', 'похожий'] ['индивидуальный', 'различие', 'характеристика', 'признак', 'психология', 'причина', 'личность']
1 ['человек', 'друг', 'группа', 'мочь', 'камень', 'понять', 'представитель'] ['психологический', 'тип', 'типология', 'характеристика', 'различие', 'группа', 'характер']
2 ['понять индивидуальный различие', 'разный ситуация', 'выделить свойство', 'собственный опыт', 'общий психология', 'среднее ребёнок', 'человек оказываться'] ['черта', 'психологический', 'характеристика', 'отличие', 'концептуализация', 'свойство', 'классификация']
3 ['маленький время опознание', 'связь скоростной характеристика', 'выше интеллект', 'интеллект', 'который', 'связать', 'работа'] ['интеллект', 'исследование', 'тип', 'показатель', 'информация', 'память', 'характеристика']
4 ['разный автор', 'темперамент', 'содержание', 'ребёнок', 'интенсивность', 'е', 'затем'] ['личность', 'темперамент', 'эмоциональность', 'поведение',

[[1, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 1, 1, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0]]

In [20]:
#RAKE
y_pred = [item for inner in rake_pred for item in inner]

m = sklearn.metrics.precision_recall_fscore_support(y_true, y_pred)
print('\t'.join([str(item) for item in m]))

[0. 1.]	[0.  0.2]	[0.         0.33333333]	[ 0 35]


##### Ошибки

* Лишние слова, которые берет во внимание - "который", "е"
* В биграммах и просто словах - повторение одних и тех же слов ("выше интеллект" и "интеллект")
* Самыйй частотные слова != ключевые

### TextRank

In [21]:
for nb,text in enumerate(textrank_res):
    for idx, words in enumerate(text):
        textrank_pred[nb][idx] = int(True in (map(lambda x: x in meta[nb], words)))
textrank_pred  

[[0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0]]

In [22]:
#text_rank
y_pred = [item for inner in textrank_pred for item in inner]
# sklearn.metrics.accuracy_score(y_true, y_pred), sklearn.metrics.recall_score(y_true, y_pred), f1_score(y_true, y_pred) \
m = sklearn.metrics.precision_recall_fscore_support(y_true, y_pred)
print('\t'.join([str(item) for item in m]))

[0. 0.]	[0. 0.]	[0. 0.]	[ 0. 35.]


c:\users\qwe\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


#### Ошибки:

* Считает биграммы подряд, в том порядке, в котором они в тексте
* В данном случае для всех биграмм частота равна 1, т.е. они встретилсь в тексте один раз, что сомнительно
* В итоговом результате - ни одного пересечения с ключевыми словами



### Итог

* Для поиска ключевых слов недостаточно знать частотность
* В исходных текстах встречалось много примеров описывамых явлений, поэтому частостность ключевых слов уменьшалась засчет преобладания других слов
* Системы не всегда считают частоты биграмм корректно, включая повторы слов или лишние слова